In [1]:
from huggingface_hub import login
from dotenv import load_dotenv
load_dotenv()

import os
token = os.getenv("HG_KEY")
login(token=token)

In [17]:
import json
import ijson
import os
import math
import time

# List of data sources
data_sources = [
    "Meidcal_Web_Corpus_en",
    "Meidcal_Web_Corpus_cn",
    "Meidcal_Literature_cn",
    "Meidcal_Literature_en",
    "Meidcal_Encyclopedia_cn",
    "Meidcal_Encyclopedia_en",
    "Meidcal_Books_cn",
    "Meidcal_Books_en",
    "SFT_data"
]

# Initialize the final dictionary
final_dict = {}

# Function to count total conversations in a JSON file
def count_conversations(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            parser = ijson.items(f, 'item')
            return sum(1 for _ in parser)  # Count all conversations
    except Exception as e:
        print(f"Error counting conversations in {file_path}: {e}")
        return 0

# Function to process a chunk of conversations (first 1%)
def process_json_chunk(file_path, num_to_include):
    conversations = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            parser = ijson.items(f, 'item')
            for i, conv in enumerate(parser):
                if i >= num_to_include:  # Stop after 1%
                    break
                # Extract the "value" fields from conversations
                turns = [turn["value"] for turn in conv["conversations"]]
                conversations.append(turns)
        return conversations
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return []

# Process each data source
used_percent = .01/100

for source in data_sources:
    if source == "SFT_data":
        file_path = os.path.join("/mnt/c/Users/HOME/Downloads", f"HuatuoGPT2-GPT4-SFT-140K.json")  # Adjust path as needed
    else:
        file_path = os.path.join("/mnt/c/Users/HOME/Downloads", f"HuatuoGPT2_Pretrain_{source}.json")  # Adjust path as needed

    if os.path.exists(file_path):
        # Count total conversations to determine 1%
        total_conversations = count_conversations(file_path)
        num_to_include = math.ceil(total_conversations * used_percent) if total_conversations > 0 else 0
        # Process only the first 1% of conversations
        if num_to_include > 0:
            final_dict[source] = process_json_chunk(file_path, num_to_include)
        else:
            final_dict[source] = []
            print(f"No conversations found in {file_path}")

# Save the combined JSON
output_path = '/mnt/c/Users/HOME/Downloads/HuatuoGPT-II/adaption/one_stage_training/train_qa_' + str(used_percent*100) + 'p.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(final_dict, f, ensure_ascii=False, indent=4)

print("Processing complete. Output saved to combined.json")

Processing complete. Output saved to combined.json


In [1]:
import json
from argostranslate import package, translate

# === 1. Cài đặt gói dịch nếu chưa có ===
package.update_package_index()
available_packages = package.get_available_packages()
zh_vi_package = next(p for p in available_packages if p.from_code == "zh" and p.to_code == "en")
package.install_from_path(zh_vi_package.download())

# === 2. Đọc file JSON gốc ===
input_path = '/mnt/c/Users/HOME/Downloads/HuatuoGPT-II/adaption/one_stage_training/train_qa_0.01p.json'
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print('loaded')

# === 3. Lặp và dịch từng câu hỏi + đáp ===
translated_data = {}

for key, qa_list in data.items():
    translated_pairs = []
    for question, answer in qa_list:
        translated_q = translate.translate(question, "zh", "en")
        translated_a = translate.translate(answer, "zh", "en")
        translated_pairs.append([translated_q, translated_a])
    # Đặt tên mới cho key
    translated_key = key.replace("_en", "_vi")
    translated_data[translated_key] = translated_pairs
    print(key)

# === 4. Ghi ra file JSON mới ===
with open(input_path[:-5] + '_vi.json', "w", encoding="utf-8") as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=2)

print("✅ Đã dịch và lưu vào output_vi.json")


Meidcal_Web_Corpus_en
Meidcal_Web_Corpus_cn
Meidcal_Literature_cn
Meidcal_Literature_en
Meidcal_Encyclopedia_cn
Meidcal_Encyclopedia_en
Meidcal_Books_cn
Meidcal_Books_en
SFT_data


NameError: name 'output_path' is not defined

In [ ]:
!cd /mnt/c/Users/HOME/Downloads/HuatuoGPT-II; python adaption/one_stage_training/data_process.py --model_path "baichuan-inc/Baichuan2-13B-Base" --data_path "adaption/one_stage_training/sample_train_qa.json"